In [1]:
from synaptics import *

In [2]:
layer = IzhikevichLayer(size=10000, preset='CH', resolution=1)
layer1 = IzhikevichLayer(size=10000, preset='CH', resolution = 1)
synapse = Synapse(layer, layer1, tau=.2)
layer.apply_current(np.ones(10000)*5)